### Acá comienza

In [95]:
import os
from glob import glob
import cv2, os, gc
from sklearn.model_selection import train_test_split

root_path = "F:/Maestria/Vision Artif/Finger-Vein-Recognition-2-master/Published_database_FV-USM_Dec2013/2nd_session/exc/"

folder_names = sorted(os.listdir(root_path))

y_list = []

In [96]:
lista = []     
x_list = []

for folder in folder_names[2:]:
    folder = folder
    if int(len(lista))==4:
        pre_lista_y = []
        for fol in lista:
            #print(lista)
            dedo = glob(os.path.join(root_path, fol, '*'))            
            imPaths = [path for path in dedo if '.jpg' in path]
            #print(imPaths)            
            for imPath in imPaths:
                image = cv2.imread(imPath, cv2.IMREAD_GRAYSCALE)
                image = cv2.resize(image, (84,84), cv2.INTER_AREA) #mandamos la imagen a cuadrado
                x_list.append(image)                
                post_fol = int(fol[4:-2])   # Hago un croping del name del folder para que no haya string y quede parte entera         
                y_list.append(post_fol)
    else:
        pass

    if int(len(lista))== 4:
        lista.clear()        
    if int(len(lista))< 4:
        lista.append(folder)
    else:
        pass    
    continue
print("FIN")

FIN


In [97]:
myset = set(y_list)
print(myset)

{42, 43, 44, 45, 46, 47, 48, 49, 50, 51}


In [98]:
len(y_list)

216

In [99]:
len(x_list)

216

In [100]:
x_list[0].shape

(84, 84)

### Importante

In [101]:
import numpy as np

a = np.array(y_list)
categories, inverse = np.unique(a, return_inverse=True)

y_list_y = np.zeros((a.size, categories.size))
y_list_y[np.arange(a.size), inverse] = 1   ## one hot encode

x_list_x = np.asarray(x_list) ## pasar x's a un array de numpy

In [102]:
image.shape

(84, 84)

In [103]:
x_list_x.shape

(216, 84, 84)

In [104]:
y_list_y.shape

(216, 10)

In [105]:
#Import Package
from __future__ import absolute_import, division, print_function, unicode_literals
#import tensorflow 
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow import keras
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from tensorflow.keras.layers import Input,Dense,Reshape,Flatten,Conv2D,Conv2DTranspose,LeakyReLU,ReLU,GlobalAveragePooling1D
from tensorflow.keras.layers import BatchNormalization,Dropout,Embedding,Activation,Concatenate
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
#from tensorflow.keras.preprocessing.image import image
from keras.utils.np_utils import to_categorical 
from tensorflow.keras import backend as K
K.clear_session()
#other library 
import os
import numpy as np
import matplotlib.pyplot as plt
import io
from PIL import Image
import math
import datetime
import pandas as pd
import random
from tensorboard.plugins.hparams import api as hp
#installation
#!pip install tensorflow-addons==0.8.3
#import tensorflow_addons as tfa
from mlxtend.plotting import plot_confusion_matrix

In [106]:
#Model 1
input_ = Input(shape=(84, 84,1))
pre_trained_model = tf.keras.applications.inception_v3.InceptionV3(input_shape = (84, 84, 1), 
                                input_tensor = input_,
                                include_top = False,
                                weights = None 
                                )
for layer in pre_trained_model.layers:
    layer.trainable = True
inception_layer = pre_trained_model.get_layer('mixed10')
inception_layer = inception_layer.output
inception_layer = layers.Flatten()(inception_layer)
print('last layer output shape: ', inception_layer.shape)

last layer output shape:  (None, 2048)


In [107]:
#Model 2
pre_trained_model_1 = tf.keras.applications.resnet50.ResNet50(input_shape = (84, 84, 1), 
                                input_tensor = input_,
                                include_top = False, 
                                weights = None 
                                )
for layer in pre_trained_model_1.layers:
    layer.trainable = True
resnet_layer = pre_trained_model_1.get_layer('conv5_block3_out')
#pre_trained_model_1.summary()
resnet_layer  = resnet_layer. output 
#x2 = layers.Conv2D(2048,(4,4),activation='relu')(x2)
resnet_layer  = layers.Flatten()(resnet_layer )
print('last layer output shape: ', resnet_layer.shape)

last layer output shape:  (None, 18432)


In [108]:
#Vision Transformer
#Tuneble Prameters
weight_decay = 0.0001
num_epochs = 150
num_classes= 10         # Tiene que la cantidad de clases de las personas
image_size = 84   ### tiene que ser cuadrado
patch_size = 6   
num_patches = (image_size // patch_size) ** 2
projection_dim = 64
num_heads = 4          
transformer_units = [
    projection_dim * 2,
    projection_dim,
] 
transformer_layers = 8
mlp_head_units = [2048, 1024]

In [109]:
def mlp(x, hidden_units, dropout_rate):
    for units in hidden_units:
        x = layers.Dense(units, activation=tf.nn.gelu)(x)
        x = layers.Dropout(dropout_rate)(x)
    return x

In [110]:
class Patches(layers.Layer):
    def __init__(self, patch_size):
        super(Patches, self).__init__()
        self.patch_size = patch_size

    def call(self, images):
        batch_size = tf.shape(images)[0]
        patches = tf.image.extract_patches(
            images=images,
            sizes=[1, self.patch_size, self.patch_size, 1],
            strides=[1, self.patch_size, self.patch_size, 1],
            rates=[1, 1, 1, 1],
            padding="VALID",
        )
        patch_dims = patches.shape[-1]
        patches = tf.reshape(patches, [batch_size, -1, patch_dims])
        return patches

In [111]:
class PatchEncoder(layers.Layer):
    def __init__(self, num_patches, projection_dim):
        super(PatchEncoder, self).__init__()
        self.num_patches = num_patches
        self.projection = layers.Dense(units=projection_dim)
        self.position_embedding = layers.Embedding(
            input_dim=num_patches, output_dim=projection_dim
        )

    def call(self, patch):
        positions = tf.range(start=0, limit=self.num_patches, delta=1)
        encoded = self.projection(patch) + self.position_embedding(positions)
        return encoded

In [112]:
patches = Patches(patch_size)(input_)
encoded_patches = PatchEncoder(num_patches, projection_dim)(patches)

for _ in range(transformer_layers):
    x1 = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
    attention_output = layers.MultiHeadAttention(
    num_heads=num_heads, key_dim=projection_dim, dropout=0.1
    )(x1, x1)
    x2 = layers.Add()([attention_output, encoded_patches])
    x3 = layers.LayerNormalization(epsilon=1e-6)(x2)
    x3 = mlp(x3, hidden_units=transformer_units, dropout_rate=0.1)
    encoded_patches = layers.Add()([x3, x2])
representation = layers.LayerNormalization(name='last_layer',epsilon=1e-6)(encoded_patches)
representation = layers.Flatten()(representation)
representation = layers.Dropout(0.5)(representation) ## quiza cambair droputs
representation= mlp(representation, hidden_units=mlp_head_units, dropout_rate=0.5)
print('last layer output shape: ', representation.shape)

last layer output shape:  (None, 1024)


In [113]:
#Merge Model
last_layer = layers.Concatenate()([inception_layer , resnet_layer, representation])
x = layers.Dense(1024, activation='relu')(last_layer)
x = tf.keras.layers.Dropout(0.2)(x)                  
x = layers.BatchNormalization(name='batch_1')(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = ReLU()(x)
x = Dense(512)(x)
x = layers.BatchNormalization(name='batch_2')(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = ReLU()(x)
x = Dense(256)(x)
x = BatchNormalization(name='batch_3')(x)
x = ReLU()(x)
x = layers.Dense(10, activation='softmax')(x) ## Densiddad de la ultiam layer tien q ser la cantidad de PERSONAS (clases)
                                                #quizas camibar activacion, normalzaiciones 
                                            

In [114]:
#Model Compile 
model = Model(inputs=input_,outputs= x)    
model.compile(optimizer = Adam(learning_rate=0.001), ## Quiza cmabir optimizer y el learning rate
              loss = tf.keras.losses.CategoricalCrossentropy(from_logits=False), 
              metrics = ['accuracy','Precision', 'Recall'])

In [115]:
x_list_x.shape

(216, 84, 84)

In [116]:
y_list_y.shape

(216, 10)

In [128]:
y_list_y

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [117]:
#Model 
history = model.fit(
            x = x_list_x,
            y = y_list_y,
            #,
            epochs = 25,
            verbose = 1,
            #callbacks=[ ], #add here
            )

Epoch 1/25
7/7 [==============================] - 37s 3s/step - loss: 2.4206 - accuracy: 0.1667 - precision: 0.4762 - recall: 0.0463
Epoch 2/25
7/7 [==============================] - 21s 3s/step - loss: 2.1305 - accuracy: 0.2824 - precision: 0.6667 - recall: 0.0833
Epoch 3/25
7/7 [==============================] - 21s 3s/step - loss: 1.6108 - accuracy: 0.4167 - precision: 0.8448 - recall: 0.2269
Epoch 4/25
7/7 [==============================] - 20s 3s/step - loss: 1.3522 - accuracy: 0.5324 - precision: 0.8495 - recall: 0.3657
Epoch 5/25
7/7 [==============================] - 21s 3s/step - loss: 1.1053 - accuracy: 0.6389 - precision: 0.8067 - recall: 0.4444
Epoch 6/25
7/7 [==============================] - 21s 3s/step - loss: 0.7839 - accuracy: 0.7269 - precision: 0.8621 - recall: 0.5787
Epoch 7/25
7/7 [==============================] - 20s 3s/step - loss: 0.4425 - accuracy: 0.8796 - precision: 0.9438 - recall: 0.7778
Epoch 8/25
7/7 [==============================] - 20s 3s/step - loss:

In [118]:
prediction = model.predict(
    x_list_x, batch_size=1, verbose='auto', steps=None, callbacks=None,
)

216/216 [==============================] - 15s 57ms/step


In [133]:
prediction[200]

array([0.00811594, 0.0100297 , 0.00157174, 0.00929043, 0.00348545,
       0.04053709, 0.47088155, 0.01255544, 0.4401481 , 0.00338458],
      dtype=float32)

### RESULTADOS - IDENTIFICACION

Como puede se observar en la imagen, se busca identificar el usuario al cual pertenece las imagenes, dandonos los porcentajes de los posibles candidatos, dandonos el usuario 48 como mayor porcentaje de probabilidad, siendo un 61%

In [ ]:
#Terminate
import os, signal
os.kill(os.getpid(), signal.SIGKILL)